In [1]:
# import libraries
import os # to interact with the file system
import numpy as np 
import pandas as pd 
import cv2  # to read the images and to do image processing 
import gc # to clear our memories
from tqdm import tqdm # to show the progress bar dynamically
from glob import glob # find the pattern of all the images



# PRE-PROCCESS THE DATA

Collect the train images

In [2]:
#collect all the images and label them into 4 labels

dirs = os.listdir('data')
images_path =[]
labels = []  

for folder in dirs:
    path = glob('./data/{}/*.jpg'.format(folder)) # extract all the images from the folder
    label = ['{}'.format(folder)] * len(path) # create a list of labels
    # append the path and label to the list
    images_path.extend(path)
    labels.extend(label)


In [3]:
# face detection
# identify the face in the image and crop the image to the face only, using pre-trained model- DNN (Deep Neural Network) in OpenCV

img_path=images_path[1]
img=cv2.imread(img_path) # read the image using open cv2

cv2.imshow('original image',img) # show the original image
cv2.waitKey(0) # wait for the key to be pressed
cv2.destroyAllWindows() # close the window



Load pre-trained Face detection model

In [4]:
# load the pre-trained model (face detection model - Caffe model)
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt','./models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

Creat func that get image, detect the faces, and return the crop faces,
using loaded face detection model - Caffe model

In [5]:
def face_detection_dnn(img):
    #calculate the blob from image (rgb mean subtraction)
    image=img.copy() # copy the image to avoid any changes to the original image
    h,w=image.shape[:2] # obtain the height and width of the image
    blob=cv2.dnn.blobFromImage(image,1,(300,300),(104.0,177.0,123.0),swapRB=True) # claculate the blob from image (rgb mean subtraction)

    # pass the blob to the face detection model and obtain the detections and predictions
    face_detection_model.setInput(blob) # pass the blob to the face detection model
    detections=face_detection_model.forward() # obtain the detections and predictions


    #print(detections.shape) # print the shape of the detections 
    #(the '200' means the number of bounding boxes detected), 7 is the number of bbox that contain the face ) 

    for i in range(0,detections.shape[2]): # loop through the detections
        confidence=detections[0,0,i,2] # obtain the confidence score for each detection
        if confidence>0.5: # filter out weak detections
            box=detections[0,0,i,3:7]*np.array([w,h,w,h]) # calculate the bounding box of the face
            box=box.astype('int') # convert the bounding box to integer
            #print(box) # print the bounding box
            pt1=(box[0],box[1]) # obtain the top left corner of the bounding box
            pt2=(box[2],box[3]) # obtain the bottom right corner of the bounding box
            #cv2.rectangle(image,pt1,pt2,(0,255,0),2) # draw the bounding box on the image
            roi = image[box[1]:box[3],box[0]:box[2]] # crop the image to the face only
            return roi
    return None


In [6]:
img_roi=face_detection_dnn(img) # obtain the cropped image
cv2.imshow('cropped image',img_roi) # show the cropped image
cv2.imshow('original image',img) # show the original image
cv2.waitKey(0) # wait for the key to be pressed
cv2.destroyAllWindows() # close the window

Creat blob from image

In [8]:
def datapreprocess(img):
#create blob from image (rgb mean subtraction)

    face = face_detection_dnn(img) # obtain the cropped image
    if face is not None: # if the face is detected

        #computing blob from image 
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True) # claculate the blob from image (rgb mean subtraction)
        blob_squeezed = np.squeeze(blob).T # squeeze the blob , reduce the dimension of the array and transpose 
        #--> Because The standard way to represent image is (height,width,channel = 3 -RGB) the blob resolt before was 4D array 
        blob_rotate = cv2.rotate(blob_squeezed,cv2.ROTATE_90_CLOCKWISE) # rotate the blob 90 degree clockwise
        blob_flip = cv2.flip(blob_rotate,1) # flip the blob horizontally

        #remove negative values and normalize the blob
        img_norm=np.maximum(blob_flip,0)/blob_flip.max() #any negative values will be set to 0 and normalize the blob

        return img_norm # return the normalized blob
    else:
        return None
    

APPLY TO ALL IMAGE AND APPEND IN A LIST

In [9]:
#apply the function to all the images

data_img =[] # create an empty list to store the processed images
label_img=[] # create an empty list to store the labels


i=0 #garbage collector counter

for path, label in tqdm(zip(images_path,labels),desc='preprocessing images'): # for loop through the image path
    img=cv2.imread(path) # read the image using open cv2
    proccess_img=datapreprocess(img) # apply the function to the image
    if proccess_img is not None: # if the image is not None
        data_img.append(proccess_img) # append the image to the list
        label_img.append(label) # append the label to the list

    
    i+=1 # increase the garbage collector counter
    if i%100 == 0: # if the garbage collector counter is equal to 100
        gc.collect() # clear the memory      

preprocessing images: 10000it [09:12, 18.11it/s]


In [10]:
x=np.array(data_img) # convert the list to numpy array
y=np.array(label_img) # convert the list to numpy array

#print(x.shape) # print the shape of the x
#print(y.shape) # print the shape of the y
np.savez('./data/data_preprocess_lite.npz',x,y) # save the data and label to the npz file